In [1]:
#1) Import all necessary packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
import ee
import geemap
import geopandas as gpd


In [2]:
import os
import sys

# Get the current working directory
current_dir = os.path.abspath('')

# Search for the 'constants.py' file starting from the current directory and moving up the hierarchy
project_root = current_dir
while not os.path.isfile(os.path.join(project_root, 'constants.py')):
    project_root = os.path.dirname(project_root)

# Add the project root to the Python path
sys.path.append(project_root)




In [3]:
# Import SHAPEFILE_PATH from constants
from constants import STUDY_BOUNDARY_PATH

# Import CHIRPS Daily: Climate Hazards Group InfraRed Precipitation With Station Data (Version 2.0 Final) and Study Boundary

https://developers.google.com/earth-engine/datasets/catalog/UCSB-CHG_CHIRPS_PENTAD

Climate Hazards Group InfraRed Precipitation with Station data (CHIRPS) is a 30+ year quasi-global rainfall dataset. CHIRPS incorporates 0.05° resolution satellite imagery with in-situ station data to create gridded rainfall time series for trend analysis and seasonal drought monitoring.

Funk, Chris, Pete Peterson, Martin Landsfeld, Diego Pedreros, James Verdin, Shraddhanand Shukla, Gregory Husak, James Rowland, Laura Harrison, Andrew Hoell & Joel Michaelsen. "The climate hazards infrared precipitation with stations-a new environmental record for monitoring extremes". Scientific Data 2, 150066. doi:10.1038/sdata.2015.66 2015.

In [ ]:
#ee.Authenticate()
ee.Initialize()

In [6]:
start_date = '2001-01-01'
end_date = '2022-12-31'
precipitation = ee.ImageCollection("UCSB-CHG/CHIRPS/DAILY").filterDate(start_date, end_date)



In [7]:
# Sort the ImageCollection by date
sorted_collection = precipitation.sort('system:time_start')

# Get the first image
first_image = sorted_collection.first()

# Get the last image
last_image = sorted_collection.sort('system:time_start', False).first()

# Print the dates of the first and last images
print('First image date:', first_image.get('system:index').getInfo())
print('Last image date:', last_image.get('system:index').getInfo())


First image date: 20010101
Last image date: 20221230


In [8]:
study_boundary = gpd.read_file(STUDY_BOUNDARY_PATH)

ee_boundary = geemap.geopandas_to_ee(study_boundary)

# Extract the first feature from the FeatureCollection
ee_boundary_feature = ee_boundary.first()

# Get the geometry of the feature
ee_boundary_geometry = ee_boundary_feature.geometry()

# Get the coordinates of the geometry
ee_boundary_coordinates = ee_boundary_geometry.coordinates().getInfo()

In [10]:
# Function to calculate ten-year average precipitation
def calculate_decadal_averages(start_year):
    end_year = start_year + 9
    start_date = ee.Date.fromYMD(start_year, 1, 1)
    end_date = ee.Date.fromYMD(end_year, 12, 31)
    decadal_precip = precipitation.filterDate(start_date, end_date).mean()
    return decadal_precip



In [11]:
# Iterate over each ten-year block and calculate averages
for year in range(2001, 2013):  # 2013 is the last start year for a ten-year block ending in 2022
    decadal_avg_precip = calculate_decadal_averages(year)
    clipped_precipitation = decadal_avg_precip.clip(ee_boundary)
    
    # Adding a suffix to the file name to indicate the specific ten-year block
    suffix = str(year) + "_" + str(year + 9)  # e.g., '2001_2010'
    
    export_params = {
        'scale': 30,  # Resolution in meters
        'region': ee_boundary_coordinates,  # Export only the region of interest
        'crs': 'EPSG:4326',  # Coordinate reference system (optional)
        'fileFormat': 'GeoTIFF',  # Export format (GeoTIFF or other supported formats)
        'fileNamePrefix': 'average_annual_precipitation_' + suffix,  # Prefix for the exported file name with suffix
    }

    export_task = ee.batch.Export.image.toDrive(
        image=clipped_precipitation,
        description='average_annual_precipitation_' + suffix,  # Include the suffix in the description as well
        folder='precipitation_data',  # Specify a folder in your Google Drive
        maxPixels=1e10,
        **export_params
    )
    # Start the export task
    export_task.start()


A new empty folder will appear in the google drive associated with Earth Engine account.
The export process takes a considerable amount of time, progress can be monitored at
https://code.earthengine.google.com/tasks

# calculate_average_precipitation

In [9]:
# Function to calculate average precipitation over the entire period
def calculate_average_precipitation(start_date, end_date):
    # Filter the ImageCollection for the specified date range
    period_precip = precipitation.filterDate(start_date, end_date)
    # Calculate the mean precipitation over the entire period
    average_precip = period_precip.mean()
    return average_precip


In [10]:

# Calculate the average precipitation from 2001 to 2022
average_precip_2001_2022 = calculate_average_precipitation(start_date, end_date)
clipped_precipitation = average_precip_2001_2022.clip(ee_boundary)

# Define the export parameters
export_params = {
    'scale': 30,  # Resolution in meters
    'region': ee_boundary_coordinates,  # Export only the region of interest
    'crs': 'EPSG:4326',  # Coordinate reference system (optional)
    'fileFormat': 'GeoTIFF',  # Export format (GeoTIFF or other supported formats)
    'fileNamePrefix': 'average_precipitation_2001_2022',  # Prefix for the exported file name
}

# Create an export task
export_task = ee.batch.Export.image.toDrive(
    image=clipped_precipitation,
    description='average_precipitation_2001_2022',  # Description of the export task
    folder='precipitation_data',  # Specify a folder in your Google Drive
    maxPixels=1e10,
    **export_params
)

# Start the export task
export_task.start()


A new empty folder will appear in the google drive associated with Earth Engine account.
The export process takes a considerable amount of time, progress can be monitored at
https://code.earthengine.google.com/tasks